In [1]:
import pandas as pd
import string
from zxcvbn import zxcvbn
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession

In [2]:
spark_available = True

In [3]:
spark = SparkSession.builder.appName('Passwords').getOrCreate()

In [4]:
def standardize_10msample(frac: float=0.1):
    sample_10m = '../data/10m_sample_common_passwords/10-million-combos.txt'
    if spark_available:
        df_10msample = spark.read.csv(sample_10m, sep='\t').sample(fraction=frac)
        # df_10msample.columns = ['username', 'password']
    
    else:
        df_10msample = pd.read_csv(sample_10m, header=None, sep='\t').astype(str).sample(frac=frac)
        df_10msample.columns = ['username', 'password']
        df_10msample.drop('username', axis=1, inplace=True)
        df_10msample['length'] = df_10msample['password'].apply(len)
        strength_features(df_10msample)
        df_10msample['class'] = df_10msample['password'].apply(class_count)
        class_expand(df_10msample)
    
    return df_10msample

In [5]:
df_10msample = standardize_10msample(frac=0.01)

In [6]:
df_10msample.describe()

DataFrame[summary: string, _c0: string, _c1: string]

In [7]:
df_10msample.columns = ['username', 'password']

AttributeError: can't set attribute